In [1]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix


from collections import defaultdict

from tqdm import tqdm
tqdm.pandas()

import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
#extract target
#then do NER with emotion
#then do classification

In [3]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_lg")

In [4]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."
text1= NER(raw_text)


In [5]:
for word in text1.ents:
    print(word.text,word.label_)

The Indian Space Research Organisation ORG
India GPE
Bengaluru GPE
Department of Space ORG
India GPE
ISRO ORG
DOS ORG


In [6]:
def ner_extract(raw_text):
    text1= NER(raw_text)
    lst = [(word.text, word.label_) for word in text1.ents]
    return lst

ner_extract(raw_text)

[('The Indian Space Research Organisation', 'ORG'),
 ('India', 'GPE'),
 ('Bengaluru', 'GPE'),
 ('Department of Space', 'ORG'),
 ('India', 'GPE'),
 ('ISRO', 'ORG'),
 ('DOS', 'ORG')]

In [7]:
os.chdir(r"D:\PACK\PhD\Study1\cleaned_dataset")
# Read in the data from the CSV file

#train_df = pd.read_pickle('df_amt_train.h5',compression='xz')

lst= ['df_amt_train.h5','df_ISOT_train.h5','df_kdd_train.h5','df_Liar_train.h5']
train_df = pd.DataFrame()
for i in lst:
    _ = pd.read_pickle(i,compression='xz')
    train_df = pd.concat([train_df,_])
 
#test_df = pd.read_pickle('df_amt_test.h5',compression='xz')
test_df = pd.DataFrame()
lst= ['df_amt_test.h5','df_ISOT_test.h5','df_kdd_test.h5','df_Liar_test.h5']

for i in lst:
    _ = pd.read_pickle(i,compression='xz')
    test_df = pd.concat([test_df,_])



d = {'not_fake':1,'fake':0}

train_df['label'] = train_df['label'].map(d)
test_df['label'] = test_df['label'].map(d)

train_df, val_df = train_test_split(train_df,
                                    test_size=0.2,
                                    random_state=0,
                                    stratify=train_df.label)
class_names = ['not_fake', 'fake']

print(train_df.shape, val_df.shape, test_df.shape)

(40500, 16) (10126, 16) (11345, 16)


In [8]:
train_df.to_pickle('train.h5',protocol=4,compression='xz')
val_df.to_pickle('val.h5',protocol=4,compression='xz')
test_df.to_pickle('test.h5',protocol=4,compression='xz')

In [9]:
train_df['NER'] = train_df.text.progress_apply(lambda x: ner_extract(x))
val_df['NER'] = val_df.text.progress_apply(lambda x: ner_extract(x))
test_df['NER'] = test_df.text.progress_apply(lambda x: ner_extract(x))

train_df

100%|████████████████████████████████████████████████████████████████████████████| 11345/11345 [13:29<00:00, 14.02it/s]


,text,label,title,subject,date,ID,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,NER
2812,California has had a movement going for the la...,0,"‘Calexit’ Is Really A Thing, Might Mean Actua...",News,"January 28, 2017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(California, GPE), (years, DATE), (Donald Tru..."
4405,Real Housewives of Orange County star Shannon ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(Orange County, GPE), (Shannon Beador, PERSON..."
16974,JOHANNESBURG (Reuters) - South African anti-ap...,1,Winnie Mandela 'in high spirits' after minor s...,worldnews,"October 19, 2017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(JOHANNESBURG, GPE), (Reuters, ORG), (South A..."
3210,Cindy Crawford’s look-alike daughter Kaia Gerb...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(Cindy Crawford, PERSON), (Kaia Gerber, PERSO..."
10524,(Reuters) - The following is the full text of ...,1,Text: Mitt Romney speech slamming Donald Trump,politicsNews,"March 3, 2016",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(Reuters, ORG), (Thursday, DATE), (Massachuse..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,"WASHINGTON (Reuters) - In late October, Presi...",1,"Trump lifts refugee ban, but admissions still ...",politicsNews,"December 8, 2017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(WASHINGTON, GPE), (Reuters, ORG), (late Octo..."
239,The American healthcare system burns 18 percen...,1,NaN,"federal-budget,health-care",NaN,4811.json,sheldon-whitehouse,U.S. Senator,Rhode Island,democrat,2.0,3.0,2.0,7.0,0.0,a speech,"[(American, NORP), (18 percent, PERCENT)]"
16834,You can believe that if this incident with Ira...,0,IRAN VIOLATES INTERNATIONAL LAW….Spineless Joh...,Government News,"Jan 15, 2016",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(Iran, GPE), (Republican, NORP), (Article 13 ..."
3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Illinois governor orders special session to br...,politicsNews,"June 15, 2017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[(CHICAGO, GPE), (Reuters, ORG), (Republican, ..."


In [14]:
def try_extract(x,loc):
    '''x[0] or x[1]'''
    try:
        return x[loc]
    except TypeError:
        return None

def process_df(_df):
    _df = _df.reset_index()
    _df = _df.explode('NER')
    _df['target'] = _df['NER'].apply(lambda x: try_extract(x,0))
    _df['target_label'] = _df['NER'].apply(lambda x: try_extract(x,1))

    ner_lst = ['PERSON','ORG','GPE','NORP','WORK_OF_ART','LOC','PRODUCT','EVENT','LAW']
    _df = _df[_df.target_label.isin(ner_lst)]
    _df = _df[['index','text','label','target','target_label']]
    return _df
    
train_df = process_df(train_df)
val_df = process_df(val_df)
test_df = process_df(test_df)

train_df

,index,text,label,target,target_label
0,2812,California has had a movement going for the la...,0,California,GPE
0,2812,California has had a movement going for the la...,0,Donald Trump,PERSON
0,2812,California has had a movement going for the la...,0,California,GPE
0,2812,California has had a movement going for the la...,0,the United States,GPE
0,2812,California has had a movement going for the la...,0,California,GPE
...,...,...,...,...,...
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Illinois,GPE
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,U.S.,GPE
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Illinois,GPE
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Medicaid,ORG


In [15]:
train_df.to_pickle('train_NER.h5',protocol=4,compression='xz')
val_df.to_pickle('val_NER.h5',protocol=4,compression='xz')
test_df.to_pickle('test_NER.h5',protocol=4,compression='xz')